In [1]:
# Importation des bibliothèques nécessaires
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import KElbowVisualizer
import numpy as np

# Charger le fichier CSV
data = pd.read_csv('nouveau_fichier.csv', delimiter=';', decimal=',')

# Indices des colonnes numériques et non numériques
numeric_cols = ['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
categorical_cols = ["type", "nameOrig", "nameDest"]

# Création du ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', StandardScaler(), numeric_cols),  # Normalisation des colonnes numériques
        ('onehot', OneHotEncoder(), ['type']),       # Encodage one-hot de la colonne 'type'
        ("cat", OneHotEncoder(), ['nameOrig', 'nameDest'])  # Encodage one-hot des colonnes 'nameOrig' et 'nameDest'
    ],
    remainder='passthrough'  # Passthrough pour les colonnes non numériques
)

# Création du pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('clustering', SpectralClustering(n_clusters=2))  # Clustering spectral avec 2 clusters par défaut
])

# Entraînement du pipeline
pipeline.fit(data)

# Définition de la liste des valeurs possibles pour le nombre de clusters
n_clusters_list = list(range(2, 4))

# Définition de la liste des scores de silhouette pour chaque valeur de n_clusters
silhouette_scores = []

# Boucle sur les valeurs possibles de n_clusters
for n_clusters in n_clusters_list:
    # Entraînement du pipeline sur les données avec le nombre de clusters spécifié
    pipeline.set_params(clustering__n_clusters=n_clusters)
    pipeline.fit(data)

    # Calcul du score de silhouette pour les clusters obtenus
    silhouette_score_ = silhouette_score(data, pipeline.named_steps['clustering'].labels_)
    silhouette_scores.append(silhouette_score_)

# Visualisation du graphique du coude (Elbow Method) pour déterminer le meilleur nombre de clusters
kelbow_visualizer = KElbowVisualizer(KMeans(), k=(2, 4), metric='silhouette', timings=False)
kelbow_visualizer.fit(data)
kelbow_visualizer.poof()

# Affichage du meilleur nombre de clusters déterminé par Elbow Method
print("Meilleur nombre de clusters déterminé par Elbow Method :", n_clusters_list[np.argmax(silhouette_scores)])

# Entraînement du pipeline sur les données avec le meilleur nombre de clusters déterminé par Elbow Method
pipeline.set_params(clustering__n_clusters=n_clusters_list[np.argmax(silhouette_scores)])
pipeline.fit(data)

# Affichage des labels des clusters obtenus
print("Labels des clusters :", pipeline.named_steps['clustering'].labels_)
